# ML Baseline

In [ ]:
# Импорты
import pandas as pd
import joblib
import numpy as np
import random
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, f1_score, hamming_loss, roc_auc_score
from typing import Dict, List, Union
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_selection import VarianceThreshold
random.seed(42)
np.random.seed(42)
pd.options.mode.chained_assignment = None

In [5]:
df = pd.read_parquet("D:/Datasets/HSE/Project/EDA/df_l_fin.parquet")

In [ ]:
df.head(2)

,author,publication_date,hubs,comments,views,url,reading_time,individ/company,bookmarks_cnt,text_length,rating_new,text_pos_tags,rating_level,text_combined,hubs_encoded
0,complex,2009-08-03 14:34:35+00:00,GTD,67,6800,https://habr.com/ru/articles/66091/,2.0,individual,25.0,2027,4.0,"[NOUN, VERB, NOUN, DET, NOUN, ADV, SCONJ, PRON...",neutral,['лишать' 'девственность' 'бложик' 'происходит...,[80]
1,popotam2,2009-07-15 20:24:31+00:00,GTD,13,3100,https://habr.com/ru/articles/64586/,1.0,individual,6.0,424,1.0,"[VERB, ADV, NUM, NOUN, VERB, PRON, ADV, ADJ, C...",neutral,['предлагать' 'вариант' 'сделать' 'организован...,[80]


In [6]:
# Выделение шкалы оценок на основе рейтинга статьи,
# Это будет таргет для предсказания оценки статьи
neg_d = df[df['rating_new']<0]['rating_new'].describe()
pos_d = df[df['rating_new']>0]['rating_new'].describe()

def rating_func(row):
    rate = row['rating_new']

    if pos_d['25%'] >= rate >= neg_d['75%']:
        return('neutral')
    
    elif pos_d['75%'] >= rate > pos_d['25%']:
        return('positive')
    elif rate > pos_d['75%']:
        return('very positive')
    
    elif neg_d['75%'] > rate >= neg_d['25%']:
        return('negative')
    elif neg_d['25%'] > rate:
        return('very negative')
    
df['rating_level'] = df.apply(rating_func, axis=1)

## Подготовка данных

In [7]:
# Объединяем текстовые токены в единую строку, 
# Это будет наш главный признак для предсказания тем (хабов) статей
df['text_combined'] = df['text_tokens'] + ' ' + df['title_tokens'] + ' ' + df['tags_tokens']

# Сразу удалим ненужные столбцы для облегчения вычислений
df = df.drop(columns=['tags_tokens', 'title_tokens', 'text_tokens']).copy()

In [8]:
# Извлекаем уникальные метки из тегов
unique_labels = set()
df['hubs'].str.split(', ').apply(unique_labels.update)  # Собираем уникальные метки
label_to_index = {label: idx for idx, label in enumerate(sorted(unique_labels))}  # Маппинг меток в индексы

# Преобразуем строки в списки индексов
df['hubs_encoded'] = df['hubs'].apply(lambda x: [label_to_index[label] for label in x.split(', ')])

In [7]:
df.head(2)

,author,publication_date,hubs,comments,views,url,reading_time,individ/company,bookmarks_cnt,text_length,rating_new,text_pos_tags,rating_level,text_combined,hubs_encoded
0,complex,2009-08-03 14:34:35+00:00,GTD,67,6800,https://habr.com/ru/articles/66091/,2.0,individual,25.0,2027,4.0,"[NOUN, VERB, NOUN, DET, NOUN, ADV, SCONJ, PRON...",neutral,['лишать' 'девственность' 'бложик' 'происходит...,[80]
1,popotam2,2009-07-15 20:24:31+00:00,GTD,13,3100,https://habr.com/ru/articles/64586/,1.0,individual,6.0,424,1.0,"[VERB, ADV, NUM, NOUN, VERB, PRON, ADV, ADJ, C...",neutral,['предлагать' 'вариант' 'сделать' 'организован...,[80]


In [9]:
# Функция для предсказания метрик precision, recall, f1-score и hamming-loss
def calculate_metrics(
    y_test: Union[List[int], np.ndarray],
    y_pred: Union[List[int], np.ndarray],
    average: str = '',
    zero_division: int = 0) -> Dict[str, float]:
    """
    Вычисление метрик precision, recall, f1-score и hamming-loss

    Параметры:
        y_test: Истинные значения
        y_pred: Предсказанные значения
        average: Метод усреднения для метрик precision, recall и f1-score
                       Может быть 'micro', 'macro'или 'weighted'
        zero_division: Обработка деления на ноль в метриках (0 или 1).

    Возвращает:
        Dict: Словарь с метриками
    """
    precision = round(precision_score(y_test, y_pred, average=average, zero_division=zero_division), 4)
    recall = round(recall_score(y_test, y_pred, average=average, zero_division=zero_division), 4)
    f1 = round(f1_score(y_test, y_pred, average=average, zero_division=zero_division), 4)
    hamming = round(hamming_loss(y_test, y_pred), 4)

    metrics = {'Precision': precision, 'Recall': recall,
               'F1-Score': f1, 'Hamming Loss': hamming}
    return metrics

**Подсказка для начинающих (для себя)**
* Микро-метрики учитывают вклад каждого класса пропорционально его частоте
* Макро-метрики вычисляются как среднее значение по всем классам без учета их частоты
* Взвешенные метрики учитывают частоту каждого класса

## Сэмпл датасета

`На 10% данных от датасета прогоним GridSearch и найдём оптимальные параметры для решения нашей задачи`

### Подготовка данных

In [10]:
# Сэмпл данных - 10% от всего объёма
sample_df = df.sample(frac=0.1, random_state=42)

# Преобразование hubs_ecoded в формат матрицы с уникальными метками хабов
mlb = MultiLabelBinarizer()
y_multi = mlb.fit_transform(sample_df['hubs_encoded'])

# Удаляем метки, которые встречаются в менее чем 1% случаев
selector = VarianceThreshold(threshold=0.01)
y_multi_reduced = selector.fit_transform(y_multi)

In [10]:
y_multi_reduced.shape

(28490, 59)

`Итого осталось 59 самых популярных хабов (тем)`

### Хабы

In [ ]:
# Подготовка данных
X = sample_df['text_combined']
y_hubs = y_multi_reduced

# Разделение на обучающую и тестовую выборки для предсказания хабов (тем) статей
X_train, X_test, y_train_h, y_test_h = train_test_split(X, y_hubs, test_size=0.25)

# Базовый пайплайн: TfidfVectorizer + OneVsRestClassifier(LogisticRegression())
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', OneVsRestClassifier(LogisticRegression(max_iter=1000, solver='sag')))])

In [ ]:
# Определение сетки параметров
params = {
     'tfidf__max_features': [2500, 5000, 10000],  # Число признаков TF-IDF
     'tfidf__ngram_range': [(1, 2)],  # Униграммы+биграммы
     'clf__estimator': [LinearSVC(), LogisticRegression(solver='sag')],  # Тип модели
     'clf__estimator__C': [0.1, 1],  # Регуляризация
}

# StratifiedKFold
cv = KFold(n_splits=3, shuffle=True, random_state=42)

# GridSearchCV
grid_search = GridSearchCV(pipeline, params, cv=cv, scoring='f1_samples', n_jobs=6, verbose=2)
grid_search.fit(X_train, y_train_h)

# Лучшая комбинация параметров
best_params_h = grid_search.best_params_
print("Лучшие параметры:", best_params_h)

# Сохранение лучшей комбинации параметров
joblib.dump(best_params_h, 'best_params_h.pkl')

Fitting 3 folds for each of 12 candidates, totalling 36 fits
Лучшие параметры: {'clf__estimator': LinearSVC(), 'clf__estimator__C': 1, 'tfidf__max_features': 10000, 'tfidf__ngram_range': (1, 2)}


In [15]:
# Оценка на тестовой выборке
y_pred_h = grid_search.best_estimator_.predict(X_test)

# Расчёт метрик для предсказания хабов (тем)
for i in ['micro', 'macro', 'weighted']:
    lr_metrics = calculate_metrics(y_pred_h, y_test_h, i)
    print(i,'-', lr_metrics)

micro - {'Precision': 0.3173, 'Recall': 0.6551, 'F1-Score': 0.4275, 'Hamming Loss': 0.0181}
macro - {'Precision': 0.2937, 'Recall': 0.6506, 'F1-Score': 0.3844, 'Hamming Loss': 0.0181}
weighted - {'Precision': 0.3795, 'Recall': 0.6551, 'F1-Score': 0.4691, 'Hamming Loss': 0.0181}


`Выводы`
* Модель показывает хороший уровень Recall (65%) на всех видах метрик, что делает ее полезной для задач, где важно не пропустить релевантные темы
* Разрыв между Macro (38%) и Weighted (47%) F1-Score говорит о том, что модель недостаточно хорошо справляется с редкими классами
* Низкое значение Hamming Loss (2%) говорит о том, что модель делает небольшое количество ошибок на уровне отдельного предсказания для каждой темы

### Рейтинг

In [17]:
# Разделение на обучающую и тестовую выборки для предсказания рейтинга статьи
y_rating = sample_df['rating_level']
y_rating = y_rating.map({'very negative': -2, 'negative': -1, 'neutral': 0, 'positive': 1, 'very positive': 2})
X_train, X_test, y_train_r, y_test_r = train_test_split(X, y_rating, test_size=0.25)

In [18]:
# Смотрим на распределение классов
y_rating.value_counts()

rating_level
 1    12282
 0     8463
 2     6417
-1      903
-2      425
Name: count, dtype: int64

`Наиболее частый класс - положительная оценка, наиболее редкий - очень негативная оценка`

In [19]:
# GridSearchCV
grid_search = GridSearchCV(pipeline, params, cv=cv, scoring='f1_weighted', n_jobs=6, verbose=2)
grid_search.fit(X_train, y_train_r)

# Лучшая комбинация параметров
best_params_r = grid_search.best_params_
print("Лучшие параметры:", best_params_r)

# Сохранение лучшей комбинации параметров
joblib.dump(best_params_r, 'best_params_r.pkl')

Fitting 3 folds for each of 12 candidates, totalling 36 fits
Лучшие параметры: {'clf__estimator': LogisticRegression(solver='sag'), 'clf__estimator__C': 1, 'tfidf__max_features': 5000, 'tfidf__ngram_range': (1, 2)}


['best_params_r.pkl']

In [20]:
# Оценка на тестовой выборке
y_pred_r = grid_search.best_estimator_.predict(X_test)

# Расчёт метрик для предсказания рейтинга
for i in ['micro', 'macro', 'weighted']:
    lr_metrics = calculate_metrics(y_pred_r, y_test_r, i)
    print(i,'-', lr_metrics)

micro - {'Precision': 0.4623, 'Recall': 0.4623, 'F1-Score': 0.4623, 'Hamming Loss': 0.5377}
macro - {'Precision': 0.2636, 'Recall': 0.2699, 'F1-Score': 0.259, 'Hamming Loss': 0.5377}
weighted - {'Precision': 0.5445, 'Recall': 0.4623, 'F1-Score': 0.4891, 'Hamming Loss': 0.5377}


`Выводы`
* Модель имеет адекватную производительность на уровне микро и взвешенных метрик для более частых классов
* Низкие микро-метрики показывают, что модель плохо работает с редкими классами (очень низкие и низкие оценки)
* Hamming Loss говорит о большом количестве ошибок (54%), что снижает практическую применимость модели

## Полный датасет

`Теперь обучим модель и посчитаем метрики для полного набора данных`

### Подготовка данных

In [21]:
# Для чистоты эксперимента уберём данные, которые были в sample_df
df = df.drop(sample_df.index).copy()

# Преобразование hubs_ecoded в формат матрицы с уникальными метками хабов
mlb = MultiLabelBinarizer()
y_multi = mlb.fit_transform(df['hubs_encoded'])

# Удаляем метки, которые встречаются в менее чем 1% случаев
selector = VarianceThreshold(threshold=0.01)
y_multi_reduced = selector.fit_transform(y_multi)

In [22]:
y_multi_reduced.shape

(256405, 58)

`Итого осталось 58 самых популярных хабов (тем)`

### Хабы

In [24]:
# Пайплайн с лучшими параметрами
pipeline_h = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', OneVsRestClassifier(LogisticRegression(max_iter=1000, solver='sag')))])

# Настройка пайплайна с лучшими параметрами
pipeline_h.set_params(**best_params_h)

# Подготовка данных
X = df['text_combined']
y_hubs = y_multi_reduced

# Разделение на обучающую и тестовую выборки для предсказания хабов (тем) статей
X_train, X_test, y_train_h, y_test_h = train_test_split(X, y_hubs, test_size=0.25)

# Обучение и предсказание по лучшему пайплайну для хабов
pipeline_h.fit(X_train, y_train_h)
y_pred_h = pipeline_h.predict(X_test)

In [25]:
# Сохранение модели для предсказания хабов (тем) статей
joblib.dump(pipeline_h, 'pipeline_h.pkl')

# Загрузка модели
#loaded_pipeline_h = joblib.load('best_pipeline_h.pkl')

# Расчёт метрик
for i in ['micro', 'macro', 'weighted']:
    lr_metrics = calculate_metrics(y_pred_h, y_test_h, i)
    print(i,'-', lr_metrics)

micro - {'Precision': 0.381, 'Recall': 0.7184, 'F1-Score': 0.4979, 'Hamming Loss': 0.0166}
macro - {'Precision': 0.3739, 'Recall': 0.7051, 'F1-Score': 0.474, 'Hamming Loss': 0.0166}
weighted - {'Precision': 0.4368, 'Recall': 0.7184, 'F1-Score': 0.5331, 'Hamming Loss': 0.0166}


`Выводы`
* Высокий Recall (~71%) указывает, что модель способна эффективно обнаруживать большинство истинных тем в статьях
* Низкий Hamming Loss (~2%) говорит о том, что в среднем модель делает мало ошибок на каждый пример, но это может быть обусловлено дисбалансом классов
* Низкий Precision (~40%) показывает, что модель склонна к ложноположительным срабатываниям
* Модель имеет потенциал, но требует доработки, особенно в части повышения точности

### Рейтинг

In [26]:
# Пайплайн с лучшими параметрами
pipeline_r = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', OneVsRestClassifier(LogisticRegression(max_iter=1000, solver='sag')))])

# Настройка пайплайна с лучшими параметрами
pipeline_r.set_params(**best_params_r)

# Разделение на обучающую и тестовую выборки для предсказания рейтинга статьи
y_rating = df['rating_level']
y_rating = y_rating.map({'very negative': -2, 'negative': -1, 'neutral': 0, 'positive': 1, 'very positive': 2})
X_train, X_test, y_train_r, y_test_r = train_test_split(X, y_rating, test_size=0.25)

In [27]:
y_rating.value_counts()

rating_level
 1    111469
 0     75879
 2     57649
-1      7672
-2      3736
Name: count, dtype: int64

`Здесь видим похожую картину как на сэмпле данных: наиболее частый класс - положительная оценка, наиболее редкий - очень негативная оценка`

In [28]:
# Обучение и предсказание по лучшему пайплайну для рейтинга
pipeline_r.fit(X_train, y_train_r)
y_pred_r = pipeline_r.predict(X_test)

In [29]:
# Сохранение модели для предсказания хабов (тем) статей
joblib.dump(pipeline_r, 'pipeline_r.pkl')

# Расчёт метрик
for i in ['micro', 'macro', 'weighted']:
    lr_metrics = calculate_metrics(y_pred_r, y_test_r, i)
    print(i,'-', lr_metrics)

micro - {'Precision': 0.4889, 'Recall': 0.4889, 'F1-Score': 0.4889, 'Hamming Loss': 0.5111}
macro - {'Precision': 0.282, 'Recall': 0.4261, 'F1-Score': 0.2807, 'Hamming Loss': 0.5111}
weighted - {'Precision': 0.5621, 'Recall': 0.4889, 'F1-Score': 0.5128, 'Hamming Loss': 0.5111}


`Выводы`
* Высокий Hamming Loss (~51%) говорит о том , что модель правильно классифицирует менее половины случаев, что недостаточно для практического применения
* Низкие макро-метрики по сравнению с взвешенными и микро-метриками указывают на то, что модель хуже справляется с редкими классами (очень низкими и низкие оценки)
* Одинаковые значения Precision, Recall и F1 (49%) в микро-метриках указывают на сбалансированную производительность по основным классам, но на недостаточно высоком уровне 
* Низкий Precision говорит о большом количестве ложноположительных предсказаний
* Текущая модель имеет ограниченную эффективность и требует доработки. Основные проблемы связаны с дисбалансом классов и неспособностью модели правильно классифицировать редкие оценки